In [1]:
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
from sklearn.metrics.pairwise import pairwise_distances
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
class KMeansClustering:
    
    def __init__(self, k, stopping_criterion="no_change") -> None:
        self.k = k
        self.stopping_criterion = stopping_criterion
        self.centroids = None
        self._sse_score = None
        self._last_sse_score = float('inf')
        self._iterations = 0
    
    def euclidean_distance(self, data_point, centroids):
        return np.sqrt(np.sum((centroids - data_point)**2, axis=1))
    
    def __sum_of_squared_errors_calc(self, centroids, data, y):
        sum_of_errors = 0.0
        for idx, d in enumerate(data):
            sum_of_errors += np.sum((centroids[y[idx]] - d) ** 2)

        return sum_of_errors

    def get_sum_of_squared_error(self):
        return self._sse_score
    def get_iterations_to_converge(self):
        return self._iterations
    
    def fit(self, X, max_iterations=200):
        self.centroids = np.random.uniform(
            low=np.amin(X, axis=0),
            high=np.amax(X, axis=0),
            size=(self.k, X.shape[1]))

        y = []
        for _ in range(max_iterations):
            y = []
            for data_point in X:

                distances = self.euclidean_distance(
                    data_point=data_point,
                    centroids=self.centroids)
                cluster_num = np.argmin(distances)
                y.append(cluster_num)
            y = np.asarray(y)

            cluster_indices = []

            for idx in range(self.k):
                cluster_indices.append(np.argwhere(y == idx))

            cluster_centers = []

            for i, indices in enumerate(cluster_indices):
                if len(indices) == 0:
                    cluster_centers.append(self.centroids[i])
                else:
                    cluster_centers.append(np.mean(X[indices], axis=0)[0])

            if self.stopping_criterion == "no_change" and np.max(self.centroids - np.array(cluster_centers)) < 1e-3:
                break
            elif self.stopping_criterion == "increase_sse":
                current_sse = self.__sum_of_squared_errors_calc(X, np.array(cluster_centers), y)
                if current_sse > self._last_sse_score:
                    break
                self._last_sse_score = current_sse
            else:
                self.centroids = np.array(cluster_centers)
            self._iterations += 1

        # Calculate the final SSE after performing K-means
        self._sse_score = self.__sum_of_squared_errors_calc(X, self.centroids, y)
        
        return y

In [3]:
data = np.array(pd.read_csv('/Users/adiaholic/Desktop/DM_HW3/kmeans_data/data.csv', header=None))
labels = np.ravel(pd.read_csv('/Users/adiaholic/Desktop/DM_HW3/kmeans_data/label.csv', header=None))
print('Data: ', data.shape)
print('Labels: ', labels.shape)

Data:  (10000, 784)
Labels:  (10000,)


In [4]:
unique_labels = np.unique(labels)
no_of_clusters = unique_labels.size
MAX_ITERATIONS = 100

In [5]:
euclidean_kmeans_m = KMeansClustering(k=no_of_clusters)
euclidean_kmeans_m_labels = euclidean_kmeans_m.fit(X=data, max_iterations=MAX_ITERATIONS)

In [6]:
np.unique(euclidean_kmeans_m_labels)

array([2, 4, 5, 6, 7, 8, 9])

In [7]:
cosine_distances = pairwise_distances(data, metric='cosine')
cosine_kmeans_m = KMeansClustering(k=no_of_clusters)
cosine_kmeans_m_labels = cosine_kmeans_m.fit(cosine_distances, max_iterations=MAX_ITERATIONS)

In [8]:
jaccard_distances = pairwise_distances(data, metric='hamming')
jaccard_kmeans_m = KMeansClustering(k=no_of_clusters)
jaccard_kmeans_m_labels = jaccard_kmeans_m.fit(X=jaccard_distances, max_iterations=MAX_ITERATIONS)

In [9]:
sse_euclidean_m = euclidean_kmeans_m.get_sum_of_squared_error()
sse_euclidean_m

65205635.80930599

In [10]:
sse_cosine_m = cosine_kmeans_m.get_sum_of_squared_error()
sse_cosine_m

3718.543362795076

In [11]:
see_jaccard_m = jaccard_kmeans_m.get_sum_of_squared_error()
see_jaccard_m

1672.8819363247092

In [12]:
sse_values = {
    'Euclidean': sse_euclidean_m,
    'Cosine': sse_cosine_m,
    'Jaccard': see_jaccard_m
}

sse_texts = [f"**SSE** of {distance} K-means = {sse}<br>" for distance, sse in sse_values.items()]

display(Markdown("".join(sse_texts)))


**SSE** of Euclidean K-means = 65205635.80930599<br>**SSE** of Cosine K-means = 3718.543362795076<br>**SSE** of Jaccard K-means = 1672.8819363247092<br>

In [13]:
def label_clusters(labels, true_labels):
    unique_labels = np.unique(true_labels)
    cluster_labels = np.zeros(len(labels), dtype=np.int)
    for cluster in range(no_of_clusters):
        cluster_indices = np.where(labels == cluster)[0]
        cluster_true_labels = true_labels[cluster_indices]
        majority_label = np.argmax([np.sum(cluster_true_labels == label) for label in unique_labels])
        cluster_labels[cluster_indices] = majority_label
    return cluster_labels

# Label clusters using majority vote
cluster_labels_euclidean = label_clusters(euclidean_kmeans_m_labels, labels)
cluster_labels_cosine = label_clusters(cosine_kmeans_m_labels, labels)
cluster_labels_jaccard = label_clusters(jaccard_kmeans_m_labels, labels)

# Compute predictive accuracy
accuracy_euclidean = accuracy_score(labels, cluster_labels_euclidean)
accuracy_cosine = accuracy_score(labels, cluster_labels_cosine)
accuracy_jaccard = accuracy_score(labels, cluster_labels_jaccard)

In [14]:
accuracy_results = [
    ("Euclidean", accuracy_euclidean),
    ("Cosine", accuracy_cosine),
    ("Jaccard", accuracy_jaccard)
]

acc_texts = [
    f"**Accuracy** of {method}-K-means = {accuracy * 100}%<br>"
    for method, accuracy in accuracy_results
]

display(Markdown(''.join(acc_texts)))


**Accuracy** of Euclidean-K-means = 54.37%<br>**Accuracy** of Cosine-K-means = 44.800000000000004%<br>**Accuracy** of Jaccard-K-means = 11.35%<br>

In [15]:
euclidean_iterations = euclidean_kmeans_m.get_iterations_to_converge()
cosine_iterations = cosine_kmeans_m.get_iterations_to_converge()
jarcard_iterations = jaccard_kmeans_m.get_iterations_to_converge()

In [16]:
text_messages = [
    f"Note: Max iterations have been set to 500 and the change in centroid position is less than 1e-3. <br>",
    f"Iterations to converge for Euclidean-K-means = **{euclidean_iterations}** <br>",
    f"Iterations to converge for Cosine-K-means = **{cosine_iterations}** <br>",
    f"Iterations to converge for Jaccard-K-means = **{jarcard_iterations}** <br>"
]

display(Markdown(''.join(text_messages)))


Note: Max iterations have been set to 500 and the change in centroid position is less than 1e-3. <br>Iterations to converge for Euclidean-K-means = **34** <br>Iterations to converge for Cosine-K-means = **27** <br>Iterations to converge for Jaccard-K-means = **1** <br>

In [17]:
euclidean_kmeans_1 = KMeansClustering(k=no_of_clusters, stopping_criterion="no_change")
euclidean_kmeans_1_predicted_labels = euclidean_kmeans_1.fit(data, max_iterations=MAX_ITERATIONS)
sse_euclidean_kmeans_1 = euclidean_kmeans_1.get_sum_of_squared_error()


In [18]:
print(f"SSE of Euclidean Kmeans when no change for centroid position is equal to {sse_euclidean_kmeans_1}")


SSE of Euclidean Kmeans when no change for centroid position is equal to 45726619.240226574


In [19]:
cosine_kmeans_1 = KMeansClustering(k=no_of_clusters, stopping_criterion="no_change")
cosine_kmeans_1_predicted_labels = cosine_kmeans_1.fit(cosine_distances, max_iterations=MAX_ITERATIONS)
sse_cosine_kmeans_1 = cosine_kmeans_1.get_sum_of_squared_error()


In [20]:
print(f"SSE of Cosine Kmeans when no change for centroid position is equal to {sse_cosine_kmeans_1}")


SSE of Cosine Kmeans when no change for centroid position is equal to 2474.2042976223465


In [21]:
jaccard_kmeans_1 = KMeansClustering(k=no_of_clusters, stopping_criterion="no_change")
jaccard_kmeans_1_predicted_labels = jaccard_kmeans_1.fit(jaccard_distances, max_iterations=MAX_ITERATIONS)
sse_jaccard_kmeans_1 = jaccard_kmeans_1.get_sum_of_squared_error()


In [22]:
print(f"SSE of Jaccard Kmeans when no change for centroid position is equal to {sse_jaccard_kmeans_1}")


SSE of Jaccard Kmeans when no change for centroid position is equal to 2202.7521173511027


In [23]:
euclidean_kmeans_2 = KMeansClustering(k=no_of_clusters, stopping_criterion="increase_sse")
euclidean_kmeans_2_predicted_labels = euclidean_kmeans_2.fit(data, max_iterations=MAX_ITERATIONS)
sse_euclidean_kmeans_2 = euclidean_kmeans_2.get_sum_of_squared_error()


In [24]:
print(f"SSE of Euclidean Kmeans when the SSE value increases in the next iteration is equal to {sse_euclidean_kmeans_2}")


SSE of Euclidean Kmeans when the SSE value increases in the next iteration is equal to 126905056.93280669


In [25]:
cosine_kmeans_2 = KMeansClustering(k=no_of_clusters, stopping_criterion="increase_sse")
cosine_kmeans_2_predicted_labels = cosine_kmeans_2.fit(cosine_distances, max_iterations=MAX_ITERATIONS)
sse_cosine_kmeans_2 = cosine_kmeans_2.get_sum_of_squared_error()
print('SSE of Cosine K-means when the SSE value increases in the next iteration =',
      sse_cosine_kmeans_2)

SSE of Cosine K-means when the SSE value increases in the next iteration = 10430.754945182669


In [26]:
print(f"SSE of Cosine Kmeans when the SSE value increases in the next iteration is equal to {sse_cosine_kmeans_2}")


SSE of Cosine Kmeans when the SSE value increases in the next iteration is equal to 10430.754945182669


In [27]:
jaccard_kmeans_2 = KMeansClustering(k=no_of_clusters, stopping_criterion="increase_sse")
jaccard_kmeans_2_predicted_labels = jaccard_kmeans_2.fit(jaccard_distances, max_iterations=MAX_ITERATIONS)
sse_jaccard_kmeans_2 = jaccard_kmeans_2.get_sum_of_squared_error()
print('SSE of Jaccard K-means when the SSE value increases in the next iteration =',
      sse_jaccard_kmeans_2)

SSE of Jaccard K-means when the SSE value increases in the next iteration = 2535.562376446988


In [28]:
euclidean_kmeans_3 = KMeansClustering(k=no_of_clusters, stopping_criterion="max_iterations")
euclidean_kmeans_3_predicted_labels = euclidean_kmeans_3.fit(data, max_iterations=MAX_ITERATIONS)
sse_euclidean_max_iteration = euclidean_kmeans_3.get_sum_of_squared_error()
print(f'SSE of Euclidean K-means when the maximum preset value {MAX_ITERATIONS} is complete =',
      sse_euclidean_max_iteration)

SSE of Euclidean K-means when the maximum preset value 100 is complete = 34761815.1094268


In [29]:
cosine_kmeans_3 = KMeansClustering(k=no_of_clusters, stopping_criterion="max_iterations")
cosine_kmeans_3_predicted_labels = cosine_kmeans_3.fit(cosine_distances, max_iterations=MAX_ITERATIONS)
sse_cosine_max_iteration = cosine_kmeans_3.get_sum_of_squared_error()
print(f'SSE of Cosine K-means when the maximum preset value {MAX_ITERATIONS} is complete =',
      sse_cosine_max_iteration)

SSE of Cosine K-means when the maximum preset value 100 is complete = 2809.4257132956236


In [30]:
jaccard_kmeans_3 = KMeansClustering(k=no_of_clusters, stopping_criterion="max_iterations")
jaccard_kmeans_3_predicted_labels = jaccard_kmeans_3.fit(jaccard_distances, max_iterations=MAX_ITERATIONS)
sse_jarcard_max_iteration = jaccard_kmeans_3.get_sum_of_squared_error()
print(f'SSE of Jaccard K-means when the maximum preset value {MAX_ITERATIONS} is complete =',
      sse_jarcard_max_iteration)

SSE of Jaccard K-means when the maximum preset value 100 is complete = 1186.5483888572712


In [31]:
table = f"""
| Algorithm | No Change in Centroid Position | SSE Value Increases in Next Iteration | Maximum Preset Value of Iterations |
|------------|--------------------------------|----------------------------------------|-------------------------------------|
| Euclidean  | {sse_euclidean_kmeans_1}              | {sse_euclidean_kmeans_2}                    | {sse_euclidean_max_iteration}|
| Jaccard    |{sse_jaccard_kmeans_1}          |{sse_jaccard_kmeans_2}                  |{sse_jarcard_max_iteration}        |
| Cosine     |{sse_cosine_kmeans_1}           |{sse_cosine_kmeans_2}                   |{sse_cosine_max_iteration}         |
"""
display(Markdown(table))



| Algorithm | No Change in Centroid Position | SSE Value Increases in Next Iteration | Maximum Preset Value of Iterations |
|------------|--------------------------------|----------------------------------------|-------------------------------------|
| Euclidean  | 45726619.240226574              | 126905056.93280669                    | 34761815.1094268|
| Jaccard    |2202.7521173511027          |2535.562376446988                  |1186.5483888572712        |
| Cosine     |2474.2042976223465           |10430.754945182669                   |2809.4257132956236         |
